In [1]:
import numpy as np
import pandas as pd

In [5]:
import chardet

with open('D:/NER_Project/ner_dataset.csv', 'rb') as f:
    result = chardet.detect(f.read(10000))  # Analyze a portion of the file
    print(result)  # This will suggest the file's encoding


{'encoding': 'ascii', 'confidence': 1.0, 'language': ''}


In [6]:
from charset_normalizer import detect

with open('D:/NER_Project/ner_dataset.csv', 'rb') as f:
    result = detect(f.read(10000))
    print(result)  # This will suggest the file's encoding


{'encoding': 'ascii', 'language': 'English', 'confidence': 0.9974}


In [7]:
import pandas as pd

# Specify the encoding when reading the CSV file
df = pd.read_csv('D:/NER_Project/ner_dataset.csv', encoding='latin1')
df

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
1048570,NaN,they,PRP,O
1048571,NaN,responded,VBD,O
1048572,NaN,to,TO,O
1048573,NaN,the,DT,O


In [8]:
import pandas as pd

# Specify the encoding when reading the CSV file
df = pd.read_csv('D:/NER_Project/ner_dataset.csv', encoding='iso-8859-1')
df

,Sentence #,Word,POS,Tag
0,Sentence: 1,Thousands,NNS,O
1,NaN,of,IN,O
2,NaN,demonstrators,NNS,O
3,NaN,have,VBP,O
4,NaN,marched,VBN,O
...,...,...,...,...
1048570,NaN,they,PRP,O
1048571,NaN,responded,VBD,O
1048572,NaN,to,TO,O
1048573,NaN,the,DT,O
